In [10]:
import torch

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Compute capability: {torch.cuda.get_device_capability(i)}")
        print(f"  Total memory: {torch.cuda.get_device_properties(i).total_memory // 1024 ** 2} MB")
else:
    print("No GPU available.")


GPU 0: NVIDIA GeForce RTX 3050 Laptop GPU
  Compute capability: (8, 6)
  Total memory: 4095 MB


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Transformations for CIFAR-10
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)  # CIFAR-10 has 10 classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

# Training the model
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

# Evaluating the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test images: {accuracy:.2f}%')

# Save the model checkpoint
torch.save(model.state_dict(), 'cnn_cifar10.pth')


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/30], Loss: 1.9080
Epoch [2/30], Loss: 1.6171
Epoch [3/30], Loss: 1.5034
Epoch [4/30], Loss: 1.4297
Epoch [5/30], Loss: 1.3756
Epoch [6/30], Loss: 1.3330
Epoch [7/30], Loss: 1.2972
Epoch [8/30], Loss: 1.2675
Epoch [9/30], Loss: 1.2438
Epoch [10/30], Loss: 1.2218
Epoch [11/30], Loss: 1.2015
Epoch [12/30], Loss: 1.1831
Epoch [13/30], Loss: 1.1662


KeyboardInterrupt: 

In [13]:
import torch
import torchvision.transforms as transforms
from PIL import Image
from io import BytesIO

# Define transformations for CIFAR-10 images
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Function to preprocess a user-uploaded image
def preprocess_user_image(image_data):
    try:
        image = Image.open(BytesIO(image_data))
        image = transform(image).unsqueeze(0)  # Add batch dimension
        return image
    except Exception as e:
        print(f"Error preprocessing image: {str(e)}")
        return None


In [14]:
import torch
import torch.nn as nn

# Define your model architecture if it's not already defined
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)  # CIFAR-10 has 10 classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load the model
model = SimpleCNN()
model.load_state_dict(torch.load('cnn_cifar10.pth'))
model.eval()


SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)

In [4]:
# Function to preprocess a user-uploaded image
def preprocess_user_image(image_data):
    try:
        image = Image.open(BytesIO(image_data))
        image = transform(image).unsqueeze(0)  # Add batch dimension
        return image
    except Exception as e:
        print(f"Error preprocessing image: {str(e)}")
        return None

# Function to handle user input image data and make predictions
def predict_user_image(image_data, model):
    try:
        # Preprocess the image
        image_tensor = preprocess_user_image(image_data)
        
        if image_tensor is None:
            return None
        
        # Make prediction
        with torch.no_grad():
            outputs = model(image_tensor)
            _, predicted = torch.max(outputs.data, 1)
            return predicted.item()
    except Exception as e:
        print(f"Error predicting image: {str(e)}")
        return None


In [15]:
# Example usage with user input
# Replace this with actual user input handling mechanism (e.g., from a web form or UI)

# Assume you have received image_data somehow (e.g., from user input or a web form)
# For demonstration, let's use a placeholder image_data (this would typically come from a different source)
image_data = open('download.jfif', 'rb').read()  # Replace 'path to your image' with the actual image data

# Predict the class of the user-provided image
predicted_class = predict_user_image(image_data, model)
if predicted_class is not None:
    print(f'Predicted class index: {predicted_class}')


Predicted class index: 5
